In [1]:
import pandas as pd
import numpy as np
from functools import reduce

In [2]:
df = pd.read_csv('datasets/san_diego_listings.csv')
df.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,6,https://www.airbnb.com/rooms/6,20240624164233,2024-06-25,city scrape,North Park Craftsmen House * Great for Families,This Beautiful Craftsmen house has all the mod...,"We have the best Neighborhood in San Diego, in...",https://a0.muscache.com/pictures/miso/Hosting-...,29,...,4.97,4.77,4.80,"STR-04545L, 631067",f,1,1,0,0,0.78
1,12447,https://www.airbnb.com/rooms/12447,20240624164233,2024-06-25,city scrape,Cozy Getaway Pacific Beach Oasis!,Your San Diego Oasis awaits!<br /><br />Privat...,NaN,https://a0.muscache.com/pictures/hosting/Hosti...,48669,...,5.00,5.00,5.00,"STR-06735L, 631362",f,2,0,2,0,0.01
2,29967,https://www.airbnb.com/rooms/29967,20240624164233,2024-06-25,city scrape,"Great home, 10 min walk to Beach","This 2 bedroom, 2 full bathroom home offers al...",NaN,https://a0.muscache.com/pictures/76103867/c9b5...,129123,...,4.93,4.82,4.77,"STR-05218L, 645110",f,5,5,0,0,0.63
3,54001,https://www.airbnb.com/rooms/54001,20240624164233,2024-06-25,city scrape,"La Jolla Cottage, 3Blks 2 Ocean; 2bd1bth, Stei...",Tucked away from city streets are 3 patio gard...,"Tranquility, stately trees, meditative, zen-fu...",https://a0.muscache.com/pictures/47035b04-633c...,252692,...,4.96,4.97,4.88,"STR-05706L, 523003",f,1,1,0,0,2.01
4,62949,https://www.airbnb.com/rooms/62949,20240624164233,2024-06-25,city scrape,Pacific Beach Cozy Private Hideaway Free Bikes,"Cozy Private Beach Retreat. 420 friendly, we h...","This is the best part of Pacific Beach, we are...",https://a0.muscache.com/pictures/hosting/Hosti...,21532,...,4.95,4.83,4.81,"STR-06358L, 62086",f,2,2,0,0,7.51


In [3]:
df.shape

(12346, 75)

In [4]:
df.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'ca

In [5]:
## drop columns that we are not using
columns_to_drop = ['listing_url', 'scrape_id', 'last_scraped', 'source', 'description', 'neighborhood_overview', 'picture_url', 
                   'host_url', 'host_location', 'host_about', 'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 
                   'host_listings_count', 'host_total_listings_count', 'host_verifications', 'host_has_profile_pic', 
                   'neighbourhood', 'neighbourhood_group_cleansed', 'minimum_minimum_nights', 'maximum_minimum_nights', 
                   'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 
                   'calendar_updated', 'has_availability', 'calendar_last_scraped', 'first_review', 'last_review', 'license', 
                   'instant_bookable', 'calculated_host_listings_count', 'calculated_host_listings_count_entire_homes',
                   'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms']
df.drop(columns=columns_to_drop, inplace=True)
df.head()

,id,name,host_id,host_name,host_since,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_identity_verified,...,number_of_reviews_ltm,number_of_reviews_l30d,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,6,North Park Craftsmen House * Great for Families,29,Sara,2008-03-03,within an hour,100%,100%,f,t,...,0,0,4.81,4.82,4.71,4.99,4.97,4.77,4.80,0.78
1,12447,Cozy Getaway Pacific Beach Oasis!,48669,Jennifer,2009-10-27,within an hour,100%,90%,t,t,...,1,1,5.00,4.00,5.00,5.00,5.00,5.00,5.00,0.01
2,29967,"Great home, 10 min walk to Beach",129123,Michael,2010-05-21,within an hour,100%,77%,t,t,...,14,0,4.77,4.74,4.57,4.93,4.93,4.82,4.77,0.63
3,54001,"La Jolla Cottage, 3Blks 2 Ocean; 2bd1bth, Stei...",252692,Marsha,2010-10-04,within a few hours,100%,78%,t,t,...,21,2,4.93,4.92,4.94,4.97,4.96,4.97,4.88,2.01
4,62949,Pacific Beach Cozy Private Hideaway Free Bikes,21532,Lisa,2009-06-14,within an hour,100%,97%,t,t,...,120,7,4.86,4.91,4.85,4.96,4.95,4.83,4.81,7.51


In [6]:
df.columns

Index(['id', 'name', 'host_id', 'host_name', 'host_since',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_identity_verified', 'neighbourhood_cleansed',
       'latitude', 'longitude', 'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'availability_30',
       'availability_60', 'availability_90', 'availability_365',
       'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'reviews_per_month'],
      dtype='object')

In [8]:
## For price column, convert to numeric only format
df['price'] = df['price'].str.replace('$', '', regex=False)
df['price'] = df['price'].str.replace(',', '', regex=False)
df['price'] = pd.to_numeric(df['price'])

In [9]:
numeric_columns = ['accommodates', 'bathrooms', 'bedrooms', 'beds', 'price', 'minimum_nights', 'maximum_nights',
                   'availability_30', 'availability_60', 'availability_90', 'availability_365', 'number_of_reviews', 
                   'number_of_reviews_ltm', 'number_of_reviews_l30d', 'review_scores_rating', 'review_scores_accuracy',
                   'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 
                   'review_scores_location', 'review_scores_value', 'reviews_per_month']
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [10]:
rows_with_missing = df[df.isna().any(axis=1)]

# Display the rows with missing values
print(rows_with_missing)

                        id                                               name   
5                    75668                           San Diego Living by JA 2  \
7                    79300                           San Diego living by JA 1   
9                   103841    Tranquil, Private Bedroom/Bath in Carmel Valley   
29                  323773             Little Italy Penthouse w/Private Deck!   
35                  394524         Private Studio a block away from the beach   
...                    ...                                                ...   
12341  1184945374539432667                        Mid-Century Garden Bungalow   
12342  1185098839763009821                   OCEAN FRONT Beachfront Studio #4   
12343  1185124700746569664                          Bright downtown view Room   
12344  1185125096110084728                    3 Bedrooms & 1 private bathroom   
12345  1185207133224082031  5th Ave Gem Chic Stay in Heart of City Free Pa...   

         host_id   host_nam

In [11]:
## drop rows that has missing values
df_cleaned = df.dropna()
df_cleaned.shape

(8893, 39)

In [12]:
missing_data_summary = df_cleaned.isnull().sum()
print("Missing data summary:\n", missing_data_summary)

Missing data summary:
 id                             0
name                           0
host_id                        0
host_name                      0
host_since                     0
host_response_time             0
host_response_rate             0
host_acceptance_rate           0
host_is_superhost              0
host_identity_verified         0
neighbourhood_cleansed         0
latitude                       0
longitude                      0
property_type                  0
room_type                      0
accommodates                   0
bathrooms                      0
bathrooms_text                 0
bedrooms                       0
beds                           0
amenities                      0
price                          0
minimum_nights                 0
maximum_nights                 0
availability_30                0
availability_60                0
availability_90                0
availability_365               0
number_of_reviews              0
number_of_reviews_lt

In [13]:
## sort data with id
df_sorted = df_cleaned.sort_values(by='id')
df_sorted.head()

,id,name,host_id,host_name,host_since,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_identity_verified,...,number_of_reviews_ltm,number_of_reviews_l30d,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,6,North Park Craftsmen House * Great for Families,29,Sara,2008-03-03,within an hour,100%,100%,f,t,...,0,0,4.81,4.82,4.71,4.99,4.97,4.77,4.80,0.78
1,12447,Cozy Getaway Pacific Beach Oasis!,48669,Jennifer,2009-10-27,within an hour,100%,90%,t,t,...,1,1,5.00,4.00,5.00,5.00,5.00,5.00,5.00,0.01
2,29967,"Great home, 10 min walk to Beach",129123,Michael,2010-05-21,within an hour,100%,77%,t,t,...,14,0,4.77,4.74,4.57,4.93,4.93,4.82,4.77,0.63
3,54001,"La Jolla Cottage, 3Blks 2 Ocean; 2bd1bth, Stei...",252692,Marsha,2010-10-04,within a few hours,100%,78%,t,t,...,21,2,4.93,4.92,4.94,4.97,4.96,4.97,4.88,2.01
4,62949,Pacific Beach Cozy Private Hideaway Free Bikes,21532,Lisa,2009-06-14,within an hour,100%,97%,t,t,...,120,7,4.86,4.91,4.85,4.96,4.95,4.83,4.81,7.51


In [14]:
df_sorted.to_csv('san_diego_listing_cleaned.csv', index=False)